In [23]:
import os

import cv2
import numpy as np
from torch.ao.nn.quantized.functional import threshold


In [ ]:
import os
import cv2
import numpy as np

def split_video(video_path, output_dir):
    threshold_drop = 1  # Soglia per rilevare il calo di luminosità
    
    # Carica il video
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    
    # Creazione dei file di output, il nome deve essere "nomevideo_0.mp4" e "nomevideo_1.mp4"
    output_filename_1 = os.path.join(output_dir, os.path.basename(video_path).replace(".mp4", "_0.mp4"))
    output_filename_2 = os.path.join(output_dir, os.path.basename(video_path).replace(".mp4", "_1.mp4"))
    
    out1 = cv2.VideoWriter(output_filename_1, fourcc, fps, (frame_width, frame_height))
    out2 = None  # Inizializziamo il secondo solo dopo il taglio
    
    def get_brightness(frame):
        """Restituisce la luminosità media di un frame."""
        return np.mean(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    # Variabili di controllo
    ret, frame = cap.read()
    if not ret:
        print("Errore nel caricamento del video.")
        cap.release()
        return
    
    initial_brightness = get_brightness(frame)  # Luminosità del primo frame
    print(f"Luminosità iniziale: {initial_brightness}")
    
    out1.write(frame)
    cut_done = False  # Flag per il taglio
    buffer_frames = []  # Buffer per evitare di scrivere frame in segmenti sbagliati
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
    
        brightness = get_brightness(frame)
    
        # Se la luminosità cala oltre la soglia, spezza il video
        if not cut_done and brightness < initial_brightness - threshold_drop:
            print(f"Calo di luminosità rilevato al frame {cap.get(cv2.CAP_PROP_POS_FRAMES)}")
            
            # Chiude il primo file
            out1.release()
    
            # Scrive i frame in sospeso nel secondo file
            out2 = cv2.VideoWriter(output_filename_2, fourcc, fps, (frame_width, frame_height))
            for bf in buffer_frames:
                out2.write(bf)
    
            cut_done = True  # Imposta il flag per il secondo segmento
            buffer_frames = []  # Svuota il buffer
    
        if cut_done:
            out2.write(frame)  # Scrive i frame del secondo segmento
        else:
            buffer_frames.append(frame)  # Aggiunge i frame al buffer prima del taglio
            if len(buffer_frames) > 2:  # Mantiene solo gli ultimi 5 frame per sicurezza
                out1.write(buffer_frames.pop(0))
    
    # Rilascia le risorse
    cap.release()
    if out1:
        out1.release()
    if out2:
        out2.release()
    
    print(f"Video segmentati: {output_filename_1}, {output_filename_2}")

def process_all_videos():
    base_dir = "data/users"
    
    users = os.listdir(base_dir)
    
    for user in users:
        user_dir = os.path.join(base_dir, user)
        if not os.path.isdir(user_dir):
            continue
        
        videos_dir = os.path.join(user_dir, "videos")
        if not os.path.isdir(videos_dir):
            continue
        
        output_dir = os.path.join(user_dir, "splitted_videos")
        os.makedirs(output_dir, exist_ok=True)
        
        for video_file in os.listdir(videos_dir):
            video_path = os.path.join(videos_dir, video_file)
            if os.path.isfile(video_path):
                print(f"Processing video {video_file} for user {user}")
                split_video(video_path, output_dir)

# Run the function to process all videos
process_all_videos()


In [26]:
data_path = "data/users"

for user in os.listdir(data_path):
    user_path = os.path.join(data_path, user)
    if not os.path.isdir(user_path):
        continue

    splitted_videos_path = os.path.join(user_path, "splitted_videos")
    videos_path = os.path.join(user_path, "videos")
    if not os.path.isdir(splitted_videos_path):
        continue
        
    print(f"User: {user}, splitted videos: {len(os.listdir(splitted_videos_path))}, videos: {len(os.listdir(videos_path))}")

User: dc894f24a0678a276269c6b9d76d97edd082fa04, splitted videos: 177, videos: 99
User: 70a079525dafd20b5b53647cd4ec30431a0e4515, splitted videos: 196, videos: 99
User: f3610367bfa13249cd736e6b1fcb1a43ef5c2d27, splitted videos: 176, videos: 99
User: 53df3de1199d9c57741e6cd82b2d32d7849de64d, splitted videos: 136, videos: 99
User: d8ce0d58a299fc0a614ba8529aa131735e17ac55, splitted videos: 148, videos: 99
User: e72dc8658bfc4f68c7aad7f1a971ba44b33c5457, splitted videos: 196, videos: 99
User: ee154cdd451335215d8cd3355a7338f99a4c8dc1, splitted videos: 136, videos: 99
User: ba57c399c288a4de733054d0201d26a85cf95548, splitted videos: 191, videos: 99
User: 94fabd6c87267c6b3eec492112be83daa9a53bef, splitted videos: 187, videos: 99
User: 0ca7288b551d6863ade0cca0248bbcbe6e600fe0, splitted videos: 184, videos: 99
User: f187690ffe7729d3a894bd03b734cb7f9f57715f, splitted videos: 197, videos: 99
User: 6d390e2eda0851bac98f7e7fcacca63bbe4ce49f, splitted videos: 148, videos: 78
User: 0d890ad3184ded354855c2